In [33]:
import pandas as pd
import numpy as np

In [34]:
# import all required csv's for output
yelp_POI=pd.read_csv('Yelp_data_1EDA.csv')
TM_events=pd.read_csv('TM_data_EDA.csv')
foursq_POI=pd.read_csv('FourSq_data_EDA.csv')

In [35]:
foursq_POI.head()

,Venue,City,State,POI,Category,Address,Distance From Venue
0,Farm League Brewing,Cambridge,Ontario,NaN,NaN,NaN,NaN
1,Dickens Pub,Calgary,Alberta,NaN,NaN,NaN,NaN
2,Under the Neon Palms at the El Mocambo,Toronto,Ontario,The Grange,Museum,317 Dundas St W,759.0
3,Grand Bizarre Exhibition Place,Toronto,Ontario,NaN,NaN,NaN,NaN
4,The Starlite Room,Edmonton,Alberta,NaN,NaN,NaN,NaN


In [36]:
# fix the data frames for the output
# by dropping columns
# by re-arranging the columns
# by renaming columns names

columns_to_drop1=["Business ID","State_x","Latitude","Longitude","latitude","longitude","Address_y"]
yelp_POI.drop(columns=columns_to_drop1,inplace=True,axis=1)
desired_columns_1=['Name', 'Category','Ratings', 'Pricing','Address_x' , 'City','State_y','Distance_From_Venue',
       'Venue']
yelp_POI=yelp_POI[desired_columns_1]
column_mapping={'Address_x' :'Address',
                'State_y':'State'}
yelp_POI.rename(columns=column_mapping,inplace=True)

                
columns_to_drop_2=["Latitude","Longitude"]
TM_events.drop(columns=columns_to_drop_2,inplace=True,axis=1)
desired_columns_2=['Event_name', 'Event_type', 'Event_dates', 'Event_start_times','Last_Purchase_Date',
       'Event_time_zone', 'Venues', 'City', 'States', 'Address','Additional_info', 'Ticket_limits', 'Age_restrictions',
       'Parking', 'Accesibility']
TM_events=TM_events[desired_columns_2]


foursq_POI.reset_index(drop=True, inplace=True)
desired_columns_3=["POI","Category","Address","City","State","Distance From Venue","Venue"]
foursq_POI=foursq_POI[desired_columns_3]




In [37]:
foursq_POI.head()

,POI,Category,Address,City,State,Distance From Venue,Venue
0,NaN,NaN,NaN,Cambridge,Ontario,NaN,Farm League Brewing
1,NaN,NaN,NaN,Calgary,Alberta,NaN,Dickens Pub
2,The Grange,Museum,317 Dundas St W,Toronto,Ontario,759.0,Under the Neon Palms at the El Mocambo
3,NaN,NaN,NaN,Toronto,Ontario,NaN,Grand Bizarre Exhibition Place
4,NaN,NaN,NaN,Edmonton,Alberta,NaN,The Starlite Room


In [4]:
# import data for the recommender
venue_ratings=pd.read_csv('Venue_Reviews.csv')
#venue_ratings.head()

In [5]:
venue_ratings.rename(columns={'Event_type': 'Category'}, inplace=True)
#venue_ratings.columns

In [6]:
# import data for the recommender
POI_ratings=pd.read_csv('POI_Ratings.csv')

In [7]:
ratings_df=pd.concat([venue_ratings,POI_ratings])

In [8]:
#ratings_df.shape

In [9]:
ratings_df.drop_duplicates()#.shape

,venue,venue_id,city,reviews,user_id,user_name,ratings,Category,Last_Purchase_Date,Venue,City,State,Address,Pricing,Distance_From_Venue,Address_y,latitude,longitude
0,Farm League Brewing,Not available,Cambridge,NaN,NaN,NaN,NaN,Rock,2024-04-20 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Farm League Brewing,Not available,Cambridge,NaN,NaN,NaN,NaN,Folk,2024-05-03 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dickens Pub,SWITg8J1mXRPNFVwHZZwng,Calgary,"Good food, great service. The wing night is a ...",hHdf-AX3pAf3JP4FqcF9SQ,Kevin L.,5.0,Rock,2024-04-20 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dickens Pub,SWITg8J1mXRPNFVwHZZwng,Calgary,"Good food, great service. The wing night is a ...",hHdf-AX3pAf3JP4FqcF9SQ,Kevin L.,5.0,Metal,2024-04-21 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dickens Pub,SWITg8J1mXRPNFVwHZZwng,Calgary,"Good food, great service. The wing night is a ...",hHdf-AX3pAf3JP4FqcF9SQ,Kevin L.,5.0,Rock,2024-04-21 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172138,Taste With Andy,fKKwYA5aPGRgRTyQJuu0mw,NaN,Amazing food and I can't believe we found this...,0gtEsazPeHr5rj2IAsZePg,Jacqui L.,5.0,Thai,NaN,Sleeman Centre,Guelph,Ontario,45 Cork Street E,Information not available,304.377672,50 Woolwich St.,43.5,-80.2
172139,Taste With Andy,fKKwYA5aPGRgRTyQJuu0mw,NaN,An EXCELLENT small casual restaurant in downto...,ykCoP-PrO2eXhXLPjoH9Mw,Yana H.,5.0,Thai,NaN,Sleeman Centre,Guelph,Ontario,45 Cork Street E,Information not available,304.377672,50 Woolwich St.,43.5,-80.2
172140,Retour Bistro,0n33W6oTCTq-2eYKABtpHw,NaN,Heaven! Everything is so intricately spiced an...,jN2r6gBSnsc-T-EeGn0PSQ,Emily S.,5.0,Bistros,NaN,Sleeman Centre,Guelph,Ontario,150 Wellington Street E,Information not available,275.794026,50 Woolwich St.,43.5,-80.2
172141,Retour Bistro,0n33W6oTCTq-2eYKABtpHw,NaN,"I'm from Toronto, and was in Guelph visiting s...",FH6z9lMzt1YH4jaOkAlYmw,Gigi S.,5.0,Bistros,NaN,Sleeman Centre,Guelph,Ontario,150 Wellington Street E,Information not available,275.794026,50 Woolwich St.,43.5,-80.2


In [10]:
# drop columns not required
columns_to_drop=['Address_y','latitude','longitude']
ratings_df.drop(columns=columns_to_drop,axis=1,inplace=True)

In [11]:
ratings_df['city'].fillna(ratings_df['City'],inplace=True)

In [12]:
ratings_df=ratings_df.drop_duplicates()
ratings_df.drop(['City'],inplace=True,axis=1)

 change the NaN ratings to -1 to indicate that they are missing, and convert the user_id,user_name to string "Not available" so that there are no NaN values in the matrix 


In [13]:
ratings_df['ratings']=ratings_df['ratings'].replace(np.nan,-1)
ratings_df[['user_id','user_name']]=ratings_df[['user_id','user_name']].replace(np.nan,"Ratings not available")


# Build Recommender

In [14]:
# Create the pivot table
recommender_matrix = ratings_df.pivot_table(index='user_id', columns='Category', values='ratings')


In [15]:
user_city= user_city # filter for the ratings to only show those options
user= user_name     # select user login
matched_user=ratings_df[ratings_df['user_name']==user]
matched_id=matched_user['user_id'].unique()
#matched_id

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity matrix
similarity_matrix = cosine_similarity(recommender_matrix.fillna(0))

# Convert similarity matrix into DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=recommender_matrix.index, columns=recommender_matrix.index)


In [17]:
# Find users similar to selected user
similarities_to_given_user = similarity_df.loc[matched_id[0]].sort_values(ascending=False)
# similarities_to_given_user

In [18]:
# Return a df of similar users
similar_users_data = []

# Iterate over the index of similarities_to_given_user
for user_id in similarities_to_given_user[similarities_to_given_user != 0].index:
    # Filter ratings_df for rows with matching user IDs
    similar_user_data = ratings_df[ratings_df['user_id'] == user_id]
    
    # Append the filtered data to similar_users_data list
    similar_users_data.append(similar_user_data)

# Concatenate the filtered data into a single DataFrame
similar_users_df = pd.concat(similar_users_data)


In [19]:
# Return a df of venues not similar but as suggestions
dis_similar = []

# Iterate over the index of similarities_to_given_user
for user_id in similarities_to_given_user[similarities_to_given_user == 0].index:
    # Filter ratings_df for rows with matching user IDs
    dis_similar_data = ratings_df[ratings_df['user_id'] == user_id]
    
    # Append the filtered data to similar_users_data list
    dis_similar.append(dis_similar_data)

# Concatenate the filtered data into a single DataFrame
suggestions = pd.concat(dis_similar)

In [20]:
# Calculate average ratings by venue for dis-similar items

average_ratings_by_venue = suggestions.groupby('venue')['ratings'].mean().reset_index()
average_ratings_by_venue.columns = ['venue', 'average_ratings']  # Rename columns for clarity
average_ratings_by_venue['average_ratings'] = average_ratings_by_venue['average_ratings'].round(1) # round the value to 1 decimal

# Merge average ratings back into the original DataFrame
suggestions_with_avg_ratings = pd.merge(suggestions, average_ratings_by_venue, on='venue', how='left')
sorted_suggestions=suggestions_with_avg_ratings.sort_values(by='average_ratings', ascending=False)


In [21]:
similar_users_df=similar_users_df[similar_users_df['city']==user_city]
#similar_users_df['venue'].unique()

In [22]:
# we use the sorted suggestions to view places with higher ratings 
sorted_suggestions=sorted_suggestions[sorted_suggestions['city']==user_city]

# Recommender output

In [24]:
restaurants=[]
Events=[]
POI=[]

In [25]:
for item in similar_users_df['venue'].unique():
    for index,row in yelp_POI.iterrows():
        if item in row['Name'] and row['City']==user_city:
            restaurants.append(row)
            break
    for index1, row1 in foursq_POI.iterrows():
        poi_value = row1['POI']
        if isinstance(poi_value, str) and row1['City']==user_city and item in poi_value :
            POI.append(row1)
            break  # Stop searching after finding a match

    for index2,row2 in TM_events.iterrows() :
        if item in row2['Venues'] and row2['City']==user_city:
            Events.append(row2)
            break
                
    
    
#print(len(restaurants),len(Events),len(POI))    

In [28]:
suggested_restaurant=[]
suggested_POI=[]
suggested_events=[]

In [29]:
for items in sorted_suggestions['venue'].unique():
    for index,row in yelp_POI.iterrows():
        if items in row['Name'] and row['City']==user_city:
            suggested_restaurant.append(row)
            break
    for index1, row1 in foursq_POI.iterrows():
        poi_value = row1['POI']
        if isinstance(poi_value, str) and items in poi_value and row1['City']==user_city:
            suggested_POI.append(row1)
            break  # Stop searching after finding a match

    for index2,row2 in TM_events.iterrows() :
        if items in row2['Venues'] and row2['City']==user_city:
            suggested_events.append(row2)
            break
                
    
    
#print(len(suggested_restaurant),len(suggested_events),len(suggested_POI))  

In [32]:
# R=pd.DataFrame(restaurants)
# E=pd.DataFrame(Events)
# P=pd.DataFrame(POI)
# SR=pd.DataFrame(suggested_restaurant)
# SE=pd.DataFrame(suggested_events)
# SP=pd.DataFrame(suggested_POI)

